In [ ]:
import org.bdgenomics.adam.models.{SequenceDictionary, SequenceRecord}
import comp.bio.aging.playground.extensions._
import org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD
import org.bdgenomics.adam.rdd.read.AlignmentRecordRDD
import org.bdgenomics.formats.avro.{Contig, NucleotideContigFragment}
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions.stringSeqExtensions._
import org.apache.spark.SparkContext
import org.apache.spark.sql._

import org.bdgenomics.adam.models.{SequenceDictionary, SequenceRecord}
import comp.bio.aging.playground.extensions._
import org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD
import org.bdgenomics.adam.rdd.read.AlignmentRecordRDD
import org.bdgenomics.formats.avro.{Contig, NucleotideContigFragment}
import org.bdgenomics.adam.rdd.ADAMContext._
import comp.bio.aging.playground.extensions.stringSeqExtensions._
import org.apache.spark.SparkContext
import org.apache.spark.sql._


In [ ]:
val root = "hdfs://"//"hdfs://namenode:8020"
val assembly = s"${root}/genomes/HUMAN/26" //prefix
val genomeAdam = s"${assembly}/genome.adam" //genome release
val featuresAdam = s"${assembly}/features.adam" //features
val data = s"${root}/data"
val genes = s"${data}/genage_LAGs_list.xlsx"


root: String = hdfs://
assembly: String = hdfs:///genomes/HUMAN/26
genomeAdam: String = hdfs:///genomes/HUMAN/26/genome.adam
featuresAdam: String = hdfs:///genomes/HUMAN/26/features.adam
data: String = hdfs:///data
genes: String = hdfs:///data/genage_LAGs_list.xlsx


In [ ]:
//val gtf = s"${assembly}/gencode.v26.chr_patch_hapl_scaff.annotation.gtf"
//val anno = sparkContext.loadGtf(gtf)
//anno.saveAsParquet(featuresAdam)
val genome = sparkContext.loadParquetContigFragments(genomeAdam)
val features = sparkContext.loadParquetFeatures(featuresAdam)

genome: org.bdgenomics.adam.rdd.contig.NucleotideContigFragmentRDD =
NucleotideContigFragmentRDD(MapPartitionsRDD[18] at map at ADAMContext.scala:385,SequenceDictionary{
KI270785.1->119912
KI270754.1->40191
KI270784.1->184404
KI270916.1->184516
KI270837.1->40090
GL383527.1->164536
KI270869.1->118774
KI270769.1->120616
KI270825.1->188315
GL000250.2->4672374
KI270849.1->244917
GL383568.1->104552
KI270711.1->42210
KI270842.1->37287
chr11->135086622
KI270425.1->1884
GL000220.1->161802
KI270758.1->76752
KV575244.1->673059
KI270740.1->37240
KI270322.1->21476
KI270729.1->280839
KI270721.1->100316
KI270728.1->1872759
GL000009.2->201709
KI270760.1->109528
KI270417.1->2043
KI270810.1->374415
GL000252.2->4604811
KI270864.1->111737
KI270366.1->8320
KI270936.1->164170
KI270723.1->38115
KI270466.1->1...

In [ ]:
val datasets = s"${root}/datasets"
val excelFile = s"${datasets}/genage_LAGs_list.xlsx"
val sqlContext = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()
val genageHuman = "genage_human"
val genageModels = "genage_models"
val lags: DataFrame = sqlContext.read
  .format("com.crealytics.spark.excel")
  .option("location", excelFile)
  .option("sheetName", genageHuman)
  .option("useHeader", "true")
  .option("treatEmptyValuesAsNulls", "true")
  .option("inferSchema", "true")
  .option("addColorColumns", "true")
  .load().withColumnRenamed("entrez gene id", "entrez")

datasets: String = hdfs:///datasets
excelFile: String = hdfs:///datasets/genage_LAGs_list.xlsx
sqlContext: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@352e9512
genageHuman: String = genage_human
genageModels: String = genage_models
lags: org.apache.spark.sql.DataFrame = [GenAge ID: double, symbol: string ... 30 more fields]


In [ ]:
lags.printSchema()

root
 |-- GenAge ID: double (nullable = true)
 |-- symbol: string (nullable = true)
 |-- aliases: string (nullable = true)
 |-- name: string (nullable = true)
 |-- entrez: string (nullable = true)
 |-- uniprot: string (nullable = true)
 |-- why: string (nullable = true)
 |-- band: string (nullable = true)
 |-- location start: double (nullable = true)
 |-- location end: double (nullable = true)
 |-- orientation: string (nullable = true)
 |-- acc promoter: string (nullable = true)
 |-- acc orf: string (nullable = true)
 |-- acc cds: string (nullable = true)
 |-- references: string (nullable = true)
 |-- orthologs: string (nullable = true)
 |-- GenAge ID_color: string (nullable = true)
 |-- symbol_color: string (nullable = true)
 |-- aliases_color: string (nullable = true)
 |-- name_color: string (nullable = true)
 |-- entrez gene id_color: string (nullable = true)
 |-- uniprot_color: string (nullable = true)
 |-- why_color: string (nullable = true)
 |-- band_color: string (nullable = tru

In [ ]:
val metadata = s"${assembly}/metadata"
val entrezGenes = s"$metadata/gencode.v26.metadata.EntrezGene"
val entrezNames = s"$metadata/gencode.v26.metadata.HGNC"

metadata: String = hdfs:///genomes/HUMAN/26/metadata
entrezGenes: String = hdfs:///genomes/HUMAN/26/metadata/gencode.v26.metadata.EntrezGene
entrezNames: String = hdfs:///genomes/HUMAN/26/metadata/gencode.v26.metadata.HGNC


In [ ]:
import sqlContext.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

case class GeneName(gene: String, gene_name: String)
val namesTable = sqlContext.read.option("sep", "\t").csv(entrezNames)
  .withColumnRenamed("_c0", "gene")
  .withColumnRenamed("_c1", "gene_name")
val names = namesTable.as[GeneName]
//val names = namesTable.withColumn("value", namesTable("value").cast(DoubleType)).as[Id]

case class GeneId(gene: String, id: Int)
val idsTable = sqlContext.read.option("sep", "\t").csv(entrezGenes)
  .withColumnRenamed("_c0", "gene")
  .withColumnRenamed("_c1", "id")
val ids =   idsTable.withColumn("id", idsTable("id").cast(IntegerType)).as[GeneId]


import sqlContext.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
defined class GeneName
namesTable: org.apache.spark.sql.DataFrame = [gene: string, gene_name: string]
names: org.apache.spark.sql.Dataset[GeneName] = [gene: string, gene_name: string]
defined class GeneId
idsTable: org.apache.spark.sql.DataFrame = [gene: string, id: string]
ids: org.apache.spark.sql.Dataset[GeneId] = [gene: string, id: int]


In [ ]:
//val resultIds = lags.join(idsTable.distinct(), $"gene" === $"entrez gene id").select($"name", $"gene", $"entrez gene id")
//(resultIds.count, ids.count, lags.count)
//idsTable.show
//val selection = lags.join(idsTable, $"id" === $"entrez gene id")
//idsTable.show()
//val res1 = lags.join(idsTable, $"id" ===  $"entrez gene id")
//val res2 = idsTable.join(lags, $"id" ===  $"entrez gene id")
//(res1.count, res2.count)
//val res = lags.filter($"entrez gene id" === "6339.0")
//res.count
//val sel = lags.select($"entrez gene id")
//sel.show()
//res.count()
//lags.select($"entrez gene id").show()
//val sel = idsTable.filter($"id" === "2690.0")
//sel.count
val selection = lags.withColumn("entrez", lags("entrez").cast(IntegerType)).select($"symbol", $"entrez")
//selection.show()
val ugenes = idsTable.distinct
val uids = idsTable.select("id").distinct
val joined1 = selection.join(ugenes, $"entrez" === $"id")
val joined2 = selection.join(uids, $"entrez" === $"id")
(selection.count, joined1.count, joined2.count)

selection: org.apache.spark.sql.DataFrame = [symbol: string, entrez: int]
ugenes: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [gene: string, id: string]
uids: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string]
joined1: org.apache.spark.sql.DataFrame = [symbol: string, entrez: int ... 2 more fields]
joined2: org.apache.spark.sql.DataFrame = [symbol: string, entrez: int ... 1 more field]
res124: (Long, Long, Long) = (307,3378,306)


(307,3378,306)

In [ ]:
features.rdd.take(10).toList

res122: List[org.bdgenomics.formats.avro.Feature] = List({"featureId": null, "name": null, "source": "HAVANA", "featureType": "gene", "contigName": "chr1", "start": 11868, "end": 14409, "strand": "FORWARD", "phase": null, "frame": null, "score": null, "geneId": "ENSG00000223972.5", "transcriptId": null, "exonId": null, "aliases": [], "parentIds": [], "target": null, "gap": null, "derivesFrom": null, "notes": [], "dbxrefs": [], "ontologyTerms": [], "circular": null, "attributes": {"gene_name": "DDX11L1", "havana_gene": "OTTHUMG00000000961.2", "gene_type": "transcribed_unprocessed_pseudogene"}}, {"featureId": null, "name": null, "source": "HAVANA", "featureType": "transcript", "contigName": "chr1", "start": 11868, "end": 14409, "strand": "FORWARD", "phase": null, "frame": null, "score": n...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
features.rdd.count

res130: Long = 2830387


2830387